# CA2 - Checkers
- Student Name: **Vinicius Moura Barros**
- Student Number: **T00244396**

The checkers game has been provided by the lecturer.
Modifications were done in the last sections of the code.

Methods developed:
- state_of_play_v2 (higher value for kings)
- state_of_play_v3 (exponential value for kings)
- state_of_play_v4 (higher value for kings + higher regular piece when closed to become king)



In [2]:
class CurrentBoard:

    def __init__(self, setup_of_board=['wwww', 'wwww', 'wwww', '    ', '    ', 'bbbb', 'bbbb', 'bbbb'],
                 newEvaluation=None):
        self.board = setup_of_board
        if newEvaluation != None:
            self.state_of_play = newEvaluation
        self.state, self.evaluation = self.state_of_play(self.board)

    def new_board(self, new_setup):
        self.board = new_setup
        self.state, self.evaluation = self.state_of_play(self.board)

    def display_board(self):
        print("-----------------")
        for row in range(8):
            c = self.board[row]
            if (row % 2) == 0:
                print("| |{}| |{}| |{}| |{}|".format(c[0], c[1], c[2], c[3]))
            else:
                print("|{}| |{}| |{}| |{}| |".format(c[0], c[1], c[2], c[3]))
            print("-----------------")

    def adjacent_squares(self, position, inc=999):
        # inc = 1 for whites, inc = -1 for black, inc  =0 for kings
        # return a list of all possible positions reachable in 1 move from position

        if inc > 10:
            return self.adjacent_squares(position, inc=self.directionAt(position))
        if inc == 0:
            return self.adjacent_squares(position, inc=-1) + self.adjacent_squares(position, inc=1)

        all_moves = []
        start_row = position[0]
        start_i = position[1]

        new_row = start_row + inc
        if (new_row >= 0) and (new_row < 8):
            all_moves.append((new_row, start_i))
            if (start_row % 2) == 0:
                if (start_i < 3):
                    all_moves.append((new_row, start_i + 1))
            else:
                if (start_i > 0):
                    all_moves.append((new_row, start_i - 1))

        return all_moves

    def playingAs(self, piece):
        if piece == 'W':
            return 'w'
        if piece == "B":
            return 'b'
        return piece

    def all_possible_moves_for(self, playing_colour):  # all moves for "w" or "b"
        non_kill_boards = []
        kill_boards = []
        found_kill = False
        for row in range(8):
            for i in range(4):
                if self.playingAs(self.get_piece_at((row, i))) == playing_colour:
                    new_non_kills, new_kills = self.construct_boards_for((row, i), found_kill)
                    if len(new_kills) > 0:
                        found_kill = True
                        kill_boards += new_kills
                    else:
                        non_kill_boards += new_non_kills
        if found_kill:
            return kill_boards
        else:
            return non_kill_boards

    def get_piece_at(self, position):
        if position != None:
            return self.board[position[0]][position[1]]
        else:
            return None

    def row_for(self, position):
        return position[0]

    def i_for(self, position):
        return position[1]

    def jump_to(self, from_position, over_position):
        row_dir = self.row_for(over_position) - self.row_for(from_position)
        new_row = self.row_for(over_position) + row_dir
        if (new_row < 0) or (new_row > 7):
            return None
        if (new_row % 2) == 0:
            if (self.i_for(from_position) == self.i_for(over_position)):
                new_i = self.i_for(from_position) - 1
            else:
                new_i = self.i_for(from_position) + 1
        else:

            if (self.i_for(from_position) == self.i_for(over_position)):
                new_i = self.i_for(from_position) + 1
            else:
                new_i = self.i_for(from_position) - 1
        if (new_i >= 0) and (new_i < 4):
            return (new_row, new_i)

    def all_kills(self, start_position, current_position, kill_list):

        all_kills_from_here = self.get_adjacent_kills(current_position, kill_list)
        all_completed_moves = []
        if (all_kills_from_here == []):
            if (kill_list == []):
                return []
            else:
                return [self.board_for(start_position, current_position, kill_list)]

        for kill in all_kills_from_here:
            all_completed_moves += self.all_kills(start_position, self.jump_to(current_position, kill),
                                                  kill_list + [kill])

        return all_completed_moves

    def king_me(self, piece):
        if piece == 'w':
            return 'W'
        if piece == 'b':
            return 'B'
        return piece

    def board_for(self, from_position, to_position, kill_list):
        board_def = []
        remove_list = kill_list + [from_position]

        piece = self.get_piece_at(from_position)
        if (self.row_for(to_position) % 7) == 0:
            piece = self.king_me(piece)
        for row_index in range(8):
            row = self.board[row_index]
            for empty_square in remove_list:
                if row_index == self.row_for(empty_square):
                    row = self.replace_at(row, self.i_for(empty_square), ' ')
            if row_index == self.row_for(to_position):
               row = self.replace_at(row, self.i_for(to_position), piece)

            board_def.append(row)

        return CurrentBoard(board_def, newEvaluation=self.state_of_play)

    def move_piece_to(self, from_position, to_position):

        return self.board_for(from_position, to_position, [from_position])

    def get_adjacent_kills(self, position, kill_list):
        piece = self.get_piece_at(position)
        kills = []
        all_adjacent_squares = self.adjacent_squares(position)
        for square in all_adjacent_squares:
            if self.is_opponent_of(piece, self.get_piece_at(square)):
                if self.is_not_already_killed(square, kill_list) and self.get_piece_at(
                        self.jump_to(position, square)) == ' ':
                    kills.append(square)
        return kills

    def is_not_already_killed(self, square, list_of_positions):
        return not (square in list_of_positions)

    def is_opponent_of(self, piece1, piece2):
        return (self.playingAs(piece1) != self.playingAs(piece2)) and (piece1 != ' ') and (piece2 != ' ')

    def replace_at(self, row_as_string, index, character):
        return row_as_string[:index] + character + row_as_string[index + 1:]

    def construct_boards_for(self, from_position, found_kill):
        non_kill_boards = []
        kill_boards = []

        kill_boards += self.all_kills(from_position, from_position, [])
        if not found_kill and len(kill_boards) == 0:
            to_position_list = self.adjacent_squares(from_position)
            for new_position in to_position_list:
                if self.get_piece_at(new_position) == " ":
                    non_kill_boards.append(self.move_piece_to(from_position, new_position))

        return non_kill_boards, kill_boards

    def directionAt(self, position):
        piece = self.get_piece_at(position)
        color = self.playingAs(piece)
        inc = 0
        if piece == color:
            if piece == 'w':
                inc = 1
            else:
                inc = -1
        return inc















In [3]:
class Search_Tree_Node:
    def __init__(self, game, playing_char, ply=0, max_ply=6):
        self.children = []
        self.ply_depth = ply
        self.max_ply = max_ply
        self.value_is_assigned = False
        self.value = 999
        self.board_for_node = game
        self.board_eval = game.evaluation if playing_char == "w" else -game.evaluation
        self.playing = playing_char
        if self.board_for_node.state == "U":
            self.generate_children()
        else:
            # game is won by either black or white
            nodePlayingSameasAI = (self.ply_depth % 2 == 0)

            if nodePlayingSameasAI:
                if self.playing == self.board_for_node.state:
                    self.value = 1000
                else:
                    self.value = -1000

            else:
                if self.playing == self.board_for_node.state:
                    self.value = -1000
                else:
                    self.value = 1000

            self.value_is_assigned = True

    def max_min_value(self):
        # if no value is assigned we will assign the max(or min) of the value of the children

        if self.value_is_assigned:
            return self.value

        self.children = sorted(self.children, key=lambda x: x.max_min_value())
        self.value_is_assigned = True
        if (self.ply_depth % 2) == 0:
            self.value = self.children[-1].value
        else:
            self.value = self.children[0].value
        return self.value

    def other_char(self, current):
        if current == "w":
            return "b"
        if current == "b":
            return "w"
        return " "

    def generate_children(self):
        if self.max_ply > self.ply_depth:
            for next_move in self.board_for_node.all_possible_moves_for(self.playing):
                self.children.append(Search_Tree_Node(next_move, self.other_char(self.playing), self.ply_depth + 1))
            if len(self.children) == 0:
                self.value_is_assigned = True
                if (self.ply_depth % 2) == 0:
                    self.value = -1000000
                else:
                    self.value = 1000000


        else:
            self.value = self.board_eval
            self.value_is_assigned = True


In [4]:
def versusBots(evalf1, evalf2):
    max_ply = 6
    V1MoveNext = False
    V1isPlaying = 'b'
    print("First Eval function is playing " + V1isPlaying)
    V2isPlaying = 'w'

    if V1MoveNext:
        currentBoard = CurrentBoard(newEvaluation=evalf1)
    else:
        currentBoard = CurrentBoard(newEvaluation=evalf2)

    currentGameOver = False

    while not currentGameOver:
        if V1MoveNext:
            st = Search_Tree_Node(currentBoard, V1isPlaying, max_ply=max_ply)
            st.max_min_value()
            if len(st.children) == 0:
                currentGameOver = True
                currentBoard.state = V2isPlaying
            else:
                currentBoard = CurrentBoard(st.children[-1].board_for_node.board, evalf2)


        else:  # V2 is playing next
            st = Search_Tree_Node(currentBoard, V2isPlaying, max_ply=max_ply)
            st.max_min_value()
            if len(st.children) == 0:
                currentGameOver = True
                currentBoard.state = V1isPlaying
            else:
                currentBoard = CurrentBoard(st.children[-1].board_for_node.board, evalf1)

        currentGameOver = currentBoard.state != "U"

        print("The evaluation of the boards below is " + str(currentBoard.evaluation))
        currentBoard.display_board()
        V1MoveNext = not V1MoveNext
        if max_ply < 8:
            count = 0
            for r in currentBoard.board:
                count += r.count(' ')

            if count > 20:
                max_ply = 15;

    print("The winner is " + str(currentBoard.state))


In [5]:

def state_of_play(boardAsString):
    whites = 0
    blacks = 0
    for row in boardAsString:
        whites += row.count('w') + 2 * row.count('W')
        blacks += row.count('b') + 2 * row.count('B')

    evaluation = whites - blacks
    if whites == 0:
        return 'b', evaluation
    if blacks == 0:
        return 'w', evaluation
    return 'U', evaluation


def state_of_play_v2(boardAsString):
    """
    This first iteration just adds a multiplier for regular pieces and for kings
    Each regular piece has value of 1 and each king has value of 2.5
    When compared to previous state_of_play, this one has the king with a better value (2.5 > 2)
    """

    whites = 0
    blacks = 0

    def pieces_value_func(num_pieces):
        return 1 * num_pieces

    def kings_value_func(num_kings):
        return 2.5 * num_kings

    for row in boardAsString:
        whites += pieces_value_func(row.count('w')) + kings_value_func(row.count('W'))
        blacks += pieces_value_func(row.count('b')) + kings_value_func(row.count('B'))

    evaluation = whites - blacks
    if whites == 0:
        return 'b', evaluation
    if blacks == 0:
        return 'w', evaluation
    return 'U', evaluation


def state_of_play_v3(boardAsString):
    """
    This version is similar to v2, but instead of having a multipler for the king,
    it sets king value as a keeps the same value multiplier for a regular piece, but
    instead the number of kings is the exponent of a base 3.

    Eg: if you have 1 king -> value 3
    Eg: if you have 2 kings -> value 9


    """
    white_pieces = 0
    white_kings = 0
    black_pieces = 0
    black_kings = 0

    def pieces_value_func(num_pieces):
        return 1 * num_pieces

    def kings_value_func(num_kings):
        return 0 if num_kings is 0 else 3 ** num_kings

    for row in boardAsString:
        white_pieces += row.count('w')
        white_kings += row.count('W')
        black_pieces = row.count('b')
        black_kings = row.count('B')


    whites = pieces_value_func(white_pieces) + kings_value_func(white_kings)
    blacks = pieces_value_func(black_pieces) + kings_value_func(black_kings)


    evaluation = whites - blacks
    if whites == 0:
        return 'b', evaluation
    if blacks == 0:
        return 'w', evaluation
    return 'U', evaluation


def state_of_play_v4(boardAsString):
    """
    v4 is also an improvement of V2. But:
    - It has multiplier for regular pieces (value 1) and for kings (value 5)
    - On top of that, regular pieces close to the final rows (before became kings) will
    be more valuable than other pieces.
    During the experiment we could observe that the last 2 rows with double the value was
    enough to win a battle against v3.

    :param boardAsString:
    :return:
    """

    whites = 0
    blacks = 0
    # assumptions:
    # we know the board has 8 rows
    # we know that whites start on top, and black starts on the bottom
    # The closer a regular piece is to become king, the more value it will have
    multiplier_per_row = [1, 1, 1, 1, 1, 1, 2, 2]
    #multiplier_per_row = [1] * 8

    def pieces_value_func(num_pieces, relative_row_number:int):
        return 1 * num_pieces * multiplier_per_row[relative_row_number]

    def kings_value_func(num_kings):
        return 5 * num_kings

    for row_number, row in enumerate(boardAsString):
        # here we need to find what's the relative row number (according to the player)
        row_number_white = row_number
        row_number_black = abs(row_number - 7)

        whites += pieces_value_func(row.count('w'), row_number_white) + kings_value_func(row.count('W'))
        blacks += pieces_value_func(row.count('b'), row_number_black) + kings_value_func(row.count('B'))

    evaluation = whites - blacks
    if whites == 0:
        return 'b', evaluation
    if blacks == 0:
        return 'w', evaluation
    return 'U', evaluation


In [5]:
# Showing that v2 wins v1
versusBots(state_of_play, state_of_play_v2)

First Eval function is playing b
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | |b| |w|
-----------------
|b| |b| |b| | | |
-----------------

In [6]:
# Showing that v2 wins v1 (Inverted attempt)
versusBots(state_of_play_v2, state_of_play)

First Eval function is playing b
The evaluation of the boards below is 0.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | |b| |w|
-----------------
|b| |b| |b| | | |
---------------

KeyboardInterrupt: 

## Results of V2 vs V1
When v2 plays whites: wins.
When v2 plays blacks: "draw" (the game gets in an infinity loop and needs to be manually interrupted.)

It is possible to observe that technically it was a draw when inverting the type of pieces the versions are playing. In the end there are the same number of kings and regular pieces for each (black/white).

From this execution and based on the shape of the board, we can also observe that blacks regular pieces are closer to become kings. And this might be a hint on how we could improve the algorithm.

In [6]:
# Showing that v3 wins v2
versusBots(state_of_play_v2, state_of_play_v3)

First Eval function is playing b
The evaluation of the boards below is 0.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 8
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | |b| |w|
-----------------
|b| |b| |b| | | |
---------------

In [7]:
# Showing that v3 wins v2 (Inverted)
versusBots(state_of_play_v3, state_of_play_v2)

First Eval function is playing b
The evaluation of the boards below is 8
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 8
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | |b| |w|
-----------------
|b| |b| |b| | | |
-----------------

## Results of V3 vs V2
When v3 plays whites: win
When v3 plays blacks: loss


In [7]:
# Showing that v4 wins v3
versusBots(state_of_play_v3, state_of_play_v4)

First Eval function is playing b
The evaluation of the boards below is 8
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 8
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | |b| |w|
-----------------
|b| |b| |b| | | |
-----------------
|

In [8]:
# Showing that v4 wins v3 (Inverted)
versusBots(state_of_play_v4, state_of_play_v3)

First Eval function is playing b
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 8
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | |b| |w|
-----------------
|b| |b| |b| | | |
-----------------
|

## Results of V4 vs V3
When v4 plays whites: win
When v4 plays blacks: win